Notebook for reading and manipulating data provided in the paper *PDZ Domain Selectivity is optimized across the mouse proteome* by Stiffler et al. The MDSM model is also implemented. 

In [1]:
import os
os.chdir('E:\Ecole\Year 3\Projet 3A')

In [2]:
import pandas as pd 
import numpy as np 

The data is provided in a multi-dimensional array. We write two classes: One called Domain which holds the data particular to the domain such as the values of the $\theta_{i,p,q}$ and the threshold values. 

The other class called Data holds all the data provided in the excel file. This class contains a list of Domains as well as a list of amino acids. 

These two classes make it easy to manipulate and read data. For testing the model proposed in the paper, it is important that we extract the $\theta_{p,q}$ for each domain. This is where the class Domain will be useful 

In [3]:
class Domain:
    def __init__(self, name):
        self.name = name
        self.thresholds = None
        self.thetas = None

In [4]:
class Data:
    def __init__(self, filename):
        self.filename = filename
        temp_df = pd.read_excel(self.filename)
        self.aminoacids = [acid.encode('utf-8') for acid in list(temp_df.columns[:20])]
        self.df = temp_df.T
        self.domains = [Domain(domain.encode('utf-8')) for domain in list(self.df.columns)]
        self.names = [domain.name for domain in self.domains]
    def create_domains(self):
        for domain in self.domains:
            domain.thetas = self.df[domain.name][:100]
            domain.thetas = np.asarray(domain.thetas)
            domain.thetas = domain.thetas.reshape(5,20)
            domain.thresholds = np.asarray(self.df[domain.name][100:])  

In [5]:
PDZ_Data = Data('Data_PDZ/MDSM_01_stiffler_bis.xls')
PDZ_Data.create_domains()

Let us browse the data now using the these two classes. Lets start off with the amino acids.

In [6]:
print PDZ_Data.aminoacids

['G', 'A', 'V', 'L', 'I', 'M', 'P', 'F', 'W', 'S', 'T', 'N', 'Q', 'Y', 'C', 'K', 'R', 'H', 'D', 'E']


Let us now see what are the different PDZ domains that we have in the data.

In [7]:
print PDZ_Data.names

['Cipp (03/10)', 'Cipp (05/10)', 'Cipp (08/10)', 'Cipp (09/10)', 'Cipp (10/10)', 'D930005D10Rik (1/1)', 'Dlgh3 (1/1)', 'Dvl1 (1/1)', 'Dvl2 (1/1)', 'Dvl3 (1/1)', 'Erbin (1/1)', 'Gm1582 (2/3)', 'GRASP55 (1/1)', 'Grip1 (6/7)', 'Grip2 (5/7)', 'Harmonin (2/3)', 'HtrA1 (1/1)', 'HtrA3 (1/1)', 'Interleukin 16 (1/4)', 'LARG (1/1)', 'LIN-7A (1/1)', 'Lin7c (1/1)', 'Lnx1 (2/4)', 'Lrrc7 (1/1)', 'Magi-1 (2/6)', 'Magi-1 (4/6)', 'Magi-1 (6/6)', 'Magi-2 (5/6)', 'Magi-2 (6/6)', 'Magi-3 (2/5)', 'Magi-3 (5/5)', 'Mpp7 (1/1)', 'MUPP1 (01/13)', 'MUPP1 (05/13)', 'MUPP1 (10/13)', 'MUPP1 (11/13)', 'MUPP1 (12/13)', 'MUPP1 (13/13)', 'NHERF-1 (1/2)', 'NHERF-2 (2/2)', 'nNOS (1/1)', 'PAR-3 (3/3)', 'PAR3B (1/3)', 'PAR6B (1/1)', 'Pdlim5 (1/1)', 'Pdzk1 (1/4)', 'Pdzk1 (3/4)', 'Pdzk3 (1/1)', 'Pdzk3 (2/2)', 'Pdzk11 (1/1)', 'PDZ-RGS3 (1/1)', 'PSD95 (1/3)', 'PTP-BL (2/5)', 'SAP97 (1/3)', 'SAP97 (3/3)', 'SAP102 (3/3)', 'Scrb1 (1/4)', 'Scrb1 (2/4)', 'Scrb1 (3/4)', 'Semcap3 (1/2)', 'Shank1 (1/1)', 'Shank3 (1/1)', 'Shroom (1/1)

Let us now explore the data for a given PDZ domain, for example the 20th one. You can access the domains by their index. The 20th domain is *LIN-7A (1/1)*

In [8]:
PDZ_Data.domains[20]
print PDZ_Data.domains[20].name

LIN-7A (1/1)


In [9]:
test_domain = PDZ_Data.domains[20]
print test_domain.name

LIN-7A (1/1)


Each domain has two variables: thetas and thresholds. The thetas are the $\theta_{i,p,q}$ as mentioned in the paper, whereas the thresholds are the values used for determining whether the PDZ domain binds to a given peptide or not

In [10]:
print test_domain.thresholds

[ 7.1429  7.1429  7.3827]


The thetas form a 5X20 matrix, that is 5 positions in the C-terminal of the peptide considered and the 20 aminon acids. You can access the data for each of the positions using the index. We note that the position -4 in the peptide corresponds to the index 0, the position -3 to the index 1 and so on.

In [11]:
print test_domain.thetas[0]

[ 0.       -0.019589  0.062421  0.68493   0.49776  -0.47603   0.2779
 -1.1723   -0.44707  -0.06063  -0.23115  -0.1164   -0.1734   -0.73216
 -0.45431   0.71784   0.95649   0.37813  -0.3057   -0.28117 ]


In [12]:
test_domain.thetas.shape

(5L, 20L)

Now we move on to the peptides. There were 217 peptides which were considered in the experiment. Each of these peptides were tested and modelled against the 74 PDZ domains which we treated earlier. Once we have the data, we simply add another variable to the class PDZ_Data and we are done.

In [13]:
pep_seqs = []
pep_names = []
with open('Data_PDZ/peptides.free') as f:
    for line in f:
        x = line.split()
        pep_seqs.append(x[1])
        pep_names.append(x[0])
        

In [14]:
PDZ_Data.pep_seqs = pep_seqs
PDZ_Data.pep_names = pep_names
PDZ_Data.pep_names

['AN2',
 'APC',
 'Aquaporin4',
 'ASIC2',
 'Caspr2',
 'Cav2.2',
 'Cftr',
 'c-KIT',
 'Claudin1',
 'Cnksr2',
 'Connexin43',
 'CRIPT',
 'CtBP1',
 'Dlgap123',
 'EphA71',
 'EphB2',
 'EphrinB12',
 'ErbB4',
 'Frizzled',
 'GluR1',
 'GluR2_1',
 'GluR5_1',
 'GlycphrinC',
 'GRK6',
 'Htr2c',
 'JAM-1',
 'KIF17',
 'KIF1B',
 'Kir2.1',
 'Kv1.4',
 'Lgltminase',
 'Liprin2',
 'Megalin',
 'Mel1a/b',
 'mGluR3',
 'ctransprtr',
 'Nav1.4',
 'Nav1.5',
 'Neurxin1/2',
 'NMDAR2A',
 'NMDAR2B',
 'P2Y1',
 'Parkin',
 'PDGFR',
 'PFK-M',
 'PIX',
 'PKC',
 'PMCA1',
 'Ril',
 'Sapk3',
 'SSTR2',
 'Stargazin',
 'Syndecan1',
 'Syndecan2',
 'TAZ',
 'Trip6',
 'TRPC4',
 'AcvR1',
 'AcvR2',
 'AcvR2b',
 'Cacna1a',
 'Cav1.2',
 'Cav2.3',
 'Cav3.2',
 'ITPR3',
 'RYR2',
 'SERCA1',
 'SERCA2A',
 'SERCA3',
 'TPC1',
 'Claudin2',
 'Claudin3',
 'Claudin4',
 'Claudin5',
 'Claudin6',
 'Claudin7',
 'Claudin8',
 'Claudin9',
 'Claudin10',
 'Claudin11',
 'Claudin13',
 'Claudin14',
 'Claudin15',
 'Claudin16',
 'Claudin18',
 'Claudin19',
 'Claudin22',

Now that the peptides have been entered, we will create one-hot representations of the last 5 positions of the peptides. Then the model itself can be validated by a simple matrix product between the one-hot representation and the transpose of the $\theta$ matrix. Taking the trace of the matrix gives us the binding score $\phi_{i}$ which we can compare with the threshold for the PDZ domain.

In [17]:
test_pep = list(PDZ_Data.pep_seqs[0])
rel_pep = test_pep[5:]
pep_matrix = np.zeros((5,20), dtype=np.int)
for i in range(len(rel_pep)):
    j = PDZ_Data.aminoacids.index(rel_pep[i])
    pep_matrix[i,j] = 1
score = np.trace(np.dot(pep_matrix, test_domain.thetas.T))

In [18]:
print score

-1.88001


In [19]:
for thres in test_domain.thresholds:
    print score > thres

False
False
False


We now compare this prediction with the data provided by Stiffler et al. The name of the PDZ Domain is *LIN-7A (1/1)* and since we took the first peptide in the sequence which according to the table S2 from the Supplementary Material to the paper is derived from the protein *AN2*, we find that it is indeed true that *LIN-7A (1/1)* doesnt bind to *AN2*

To facilitate further data analysis, we create another object called Peptide which will store the name and sequence of each of the peptides in the data set. Once created, this peptide object will be added to the Data class. Thus all our data will stay in the Data class, while we can do other manipulations using the Domain or Peptide classes.

In [20]:
def evaluate_model(domain, peptide):
    test_pep = list(peptide)
    rel_pep = test_pep[5:]
    pep_matrix = np.zeros((5,20), dtype=np.int)
    for i in range(len(rel_pep)):
        j = PDZ_Data.aminoacids.index(rel_pep[i])
        pep_matrix[i,j] = 1
    score = np.trace(np.dot(pep_matrix, domain.thetas.T))
    print score
    for thres in domain.thresholds:
        print score > thres

In [21]:
print PDZ_Data.domains[16].name
print PDZ_Data.pep_names[6]
evaluate_model(PDZ_Data.domains[16], PDZ_Data.pep_seqs[9])

HtrA1 (1/1)
Cftr
15.88935
True
True
True
